
This script is for the statistical data analysis for the article "Classification and biomarker identification of prostate tissue from TRAMP mice with hyperpolarized 13C-SIRA" by A. Frahm et al. Talanta. 2021 Aug 20:122812.

All code is written by A Frahm (annetirsdag@gmail.com).

Versions used:

Python: 3.6.10 Scipy: 1.5.2 sklearn: 0.23.2


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import winsound

from collections import defaultdict #Used in RF for ordered dictionary
from scipy import stats

from sklearn import preprocessing, svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.model_selection import LeaveOneOut, cross_val_predict, cross_val_score, GridSearchCV,  StratifiedShuffleSplit
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestClassifier as RF

#set figuresize, some functions changes this
plt.rcParams['figure.figsize'] = [20, 10]

#set up colors for plots to use
color_set = np.array(["#990000","steelblue", 'indigo', 'lime', 'chocolate', 'navy'])


In [ ]:
def makepca(d, cat,  scale = "s"):    
    
    """
    Makes Principal Components Analysis (PCA) and plots score plot of 1. and 2. components. 
    Scales data first.
    Prints out list of importance (in %) of all components.

    Input:
    d(n x m pd DataFrame): X-varaible; datamatrix with features as columns and datapoints as rows.
    cat(n-lenght list-like): Y-variable; labels to color after
    scale(string, "s", "p" or "n") :scaling method. "s" = standard(default). "p" = Pareto. "n" = No scaling.
    
    """
    
    #scale data set "sf" parameter that scales spectrum in loading plot
    if scale == "s":
        #autoscale
        data =  (d - d.mean())/d.std()
    elif scale == "p":
        #paretoscale
        data =  (d - d.mean())/d.std()**0.5
    elif scale == "n":
        #no scale
        data = d
    else:
        #End function if no method chosen
        raise Exception("No correct scale method specified: 'scale' has to be 's','p' or 'n'")
    
    
    #check if number of components have been chosen

    #make PCA    
    pca = PCA()
    pca.fit(data)
    data_pc = pca.transform(data)
    
    plt.rcParams['figure.figsize'] = [10, 10]
    
    
    #get classes as numbers
    le = preprocessing.LabelEncoder()
    le.fit(cat)
    label_set = le.classes_
    labels_data =le.transform(cat)
        
    
    #Prepare colors corresponding to labels for plotting
    colors = {}
    for i in range(len(label_set)):
        colors[label_set[i]] = color_set[i] #color_set has to be set outside of function
    
    types = len(label_set)
    
    #Plot PCA scores
    for label in range(len(label_set)):
        
        c = colors[label_set[label]]
        x = data_pc[:,0][labels_data == label]
                
        y=data_pc[:,1][labels_data == label]
        
        plt.scatter(x,y, color = c, label = label_set[label], s= 70)
        
    bob = "PC1 %.2f%%" %(pca.explained_variance_ratio_[0]*100)
    plt.xlabel(bob, fontsize = 25)
    bob = "PC2 %.2f%%" %(pca.explained_variance_ratio_[1]*100)
    plt.ylabel(bob, fontsize = 25)
    plt.title('PCA Scoreplot', fontsize = 25)
    plt.legend( prop={'size': 25}, loc = 2)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.show()
    
    with np.printoptions(precision=3, suppress=True):
        print(np.round(pca.explained_variance_ratio_*100, 2))



In [2]:
def makerf(data, cat, trees = 600, loops = 10000, params = None):
    
    """
    Function for making importance ranking of features with random forest classification.
    Importance measured through shuffling values in each feature and comparing classifictaion success 
    between normal and shuffled data.
    
    Input:
    data(dataframe, m x n): x-variable, data with features as columns and datapoints as rows.
    cat(list-like, n): y-variable to classify after.
    trees(optional) = n_estismaters for forest, default 600
    loops(optional) = Number of repetitions for 
    params(dict)(optional): additional parameters for random forest if non-default to be used.
    
    Output:
    imp(dataframe): Importance matrix with columns "Mean" and "std err."
    
    """
    
    #make sure we can handle Y in both array and Dataframe
    if type(cat) == pd.Series:
        cat = np.ravel(cat.values)
    
    #define Random Forest classifier and fit to all data
    if params is None:
        rf = RF(n_estimators= trees, oob_score= True)
    else:
        rf = RF(n_estimators= trees, oob_score= True, **params)
    
    rf.fit(data,cat)
    
    print("Out-of-bag score (%):", rf.oob_score_*100)
    
    scores = defaultdict(list) #empty library for tracking scores

    #define train-test splits, stratified to ensure all cell lines are in test
    splits = StratifiedShuffleSplit(loops, train_size = 0.7)

    #run test 
    for train_idx, test_idx in splits.split(data, cat):
        
        #sort training and testing
        X_train = data.values[train_idx]
        X_test = data.values[test_idx]
        Y_train = cat[train_idx]
        Y_test = cat[test_idx]
    
        #fit RF to training data
        r = rf.fit(X_train, Y_train)
        
        #get true accuracy
        acc = sum(rf.predict(X_test) == Y_test)/len(Y_test)
    
        #for each feature get difference in accuracy when test classes are shuffled
        if acc > 0: #avoid divide by zero error, sometimes occurying with small dataset/randomized data
            for i in range(len(data.columns)):
                X_t = X_test.copy()
                np.random.shuffle(X_t[:, i])
                shuff_acc = sum(rf.predict(X_t) == Y_test)/len(Y_test)
                scores[data.columns[i]].append((acc-shuff_acc)/acc)

        
    imp = pd.DataFrame(columns= ['Mean', 'std err.', 'color'])

    #color code positive-negative
    for feat in scores: 
        m = np.mean(scores[feat])
        c = 'r'
        if m > 0:
            c = 'g'
        imp.loc[feat] = [m, stats.sem(scores[feat]), c] 
        #stats.sem = standard error on the mean

    imp=imp.sort_values('Mean', ascending = False)
    
    #plot important features, maximum 30
    ml = min(len(imp), 30)
    
    imp.iloc[:ml].plot.bar(y = 'Mean', yerr = 'std err.', color = imp.color, legend = False)
    plt.ylabel('Relative importance ')
    plt.show()
    return(imp)

In [ ]:
def imp_clean(imp):
    """ 
    Takes RF importance results and returns only significant features.
    Significance cutoff set to be 5% of score of highest importance feature.
    
    Input:
    imp(dataframe): Importance matrix with columns "Mean" and "std err."
    
    Output:
    imp_sig(array): Index values of features found to have significant positive importance.
    
    """
    
    cutoff = (imp.Mean.iloc[0] - imp['std err.'].iloc[0]) * 0.05
    
    imp_sig = imp[(imp.Mean - imp['std err.']) > cutoff]
    
    return(imp_sig.index.values)

In [ ]:
def plot_rfs(imp_chart, ml):
    
    """
    Makes pretty plot of only the significantly important features.
    
    Input:
    imp_chart(dataframe): Importance matrix with columns "Mean" and "std err."
    ml(array): List of features in imp_chart to be plotted
    """
    
    imps = imp_chart.loc[ml]
    
    plt.rcParams['figure.figsize'] = [10, 10]
    
    imps.plot.bar(y = 'Mean', yerr = 'std err.', color = 'olivedrab', legend = False)
    plt.title('RF feature importance', fontsize = 25)
    plt.ylabel('Relative importance', fontsize = 25)
    plt.yticks(fontsize = 20)
    plt.xlabel('Chemical shift (ppm)', fontsize = 25)
    plt.xticks(fontsize = 20)
    plt.show()

In [1]:
def loadings_pca(d, annotate = False, scale = "s"):

    """
    Makes Principal Components Analysis (PCA) and plots loading plot of 1. and 2. components, 
    for all features in the dataset. 
    Scales data first.

    Input:
    d(n x m pd DataFrame): X-varaible; datamatrix with features as columns and datapoints as rows.
    annotate(default = False): Boolean, wheter to print feature names in plot. 
    scale(string, "s", "p" or "n") :scaling method. "s" = standard(default). "p" = Pareto. "n" = No scaling.
    
    """
    
    
    #scale data set "sf" parameter that scales spectrum in loading plot
    if scale == "s":
        #autoscale
        data =  (d - d.mean())/d.std()
    elif scale == "p":
        #paretoscale
        data =  (d - d.mean())/d.std()**0.5
    elif scale == "n":
        #no scale
        data = d
    else:
        #End function if no method chosen
        raise Exception("No correct scale method specified: 'scale' has to be 's','p' or 'n'")
    
    
    
    #make PCA    
    pca = PCA(n_components=2)
    pca.fit(data)
    data_pc = pca.transform(data)        
    
    
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    
    #Plot PCA scores
    
    plt.rcParams['figure.figsize'] = [10, 10]  
    
    for i, feature in enumerate(d.columns):
        
        x = loadings[i, 0]
        y = loadings[i, 1]
        
        plt.plot([0, x], [0, y], 'k-', lw=2)
        
        if annotate == True:
            plt.annotate(feature, (x, y), fontsize = 15)
    
    bob = "PC1 %.2f%%" %(pca.explained_variance_ratio_[0]*100)
    plt.xlabel(bob, fontsize = 25)
    bob = "PC2 %.2f%%" %(pca.explained_variance_ratio_[1]*100)
    plt.ylabel(bob, fontsize = 25)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.title('PCA Loadings', fontsize = 25)
    #plt.xlim([-0.25, 1])
    #plt.ylim([-0.85, 0.41])
    
    plt.show()

In [ ]:
def makesvm(d, cat, scale = "s", print_out = None):
    """
    Function for making Support Vector Machine (svm) classification, using grid search to optimize internal parameters.
    Data can be scaled before algoritm is run, different scaling methods can be chosen
    Options for cost parameter (C) are 2^n with n being (-5:5). A linear kernel function is used.
    
    Input:
    d(n x m pd DataFrame): X-varaible; datamatrix with features as columns and datapoints as rows.
    cat(n-lenght list-like): Y-variable; labels to classify after
    scale(string, "s", "p" or "n") :scaling method. "s" = standard(default). "p" = Pareto. "n" = No scaling.
    print_out(bool): If not "None" stats on classification parameters and errors are printed on screen.
    
    Output:
    grid(GridSearchCV object): Obtimizing algoritm fitted to the data. 
    
    """
    
    
    #scale data
    if scale == "s":
        #Autoscale
        data =  (d - d.mean())/d.std()
    elif scale == "p":
        #Pareto scale
        data =  (d - d.mean())/d.std()**0.5
    elif scale == "n":
        #No scaling
        data = d
    else:
        raise Exception("No correct scale method specified: 'scale' has to be 's','p' or 'n'")
    
    #get classes as numbers
    le = preprocessing.LabelEncoder()
    le.fit(cat)
    label_set = le.classes_ #list of classes
    labels_data =le.transform(cat) #encoded y-variable
    
    #set up standard SVM
    clf = svm.SVC(probability=True)
    loo = LeaveOneOut()
    
    
    #set up options for parameter grid
    bob = np.arange(-5, 5, 1)
    bub = np.ones(len(bob))*2
    power2 = bub**bob
    
    #param_grid = [{'kernel': ['linear'], 'C': power2},{'kernel': ('rbf', 'poly'), 'C': power2,  'gamma': power2}]
    param_grid = [{'kernel': ['linear'], 'C': power2}]    
    
    #set up gridsearch
    grid = GridSearchCV(clf, param_grid, refit = True, cv= loo) 
  
    # fitting the model for grid search 
    grid.fit(data, cat) 
    
    #print some nice stats if wanted
    if print_out is not None:
        
        #refit SVM classifier. If grid is used directly, predicted will be wrong.
        clf = svm.SVC(**grid.best_params_, probability=True)
        clf.fit(data, labels_data)
        predicted_loo = cross_val_predict(clf, data, labels_data, cv= loo)
                  
        acc_loo = grid.best_score_ * 100
        params = grid.best_params_
        
        #make confusion matrix
        bub = np.array([label_set[s] for s in labels_data])
        bob = np.array([label_set[s] for s in predicted_loo])
        con_loo = pd.crosstab(bub, bob, rownames= ["Actual"], colnames= ["predicted loo"])
        
        #make list of errors 
        loo_pred_err = pd.DataFrame(np.column_stack([bub, bob]), columns = ["Actual", "Predicted"], index= d.index)
        loo_pred_err = loo_pred_err[loo_pred_err.Actual != loo_pred_err.Predicted]
        
        print("Leave-One-Out validated classification score: ", acc_loo)
        print("Parameters used: ", params)
        print("Classification errors:")
        print(con_loo)
        print(loo_pred_err)
        
    return(grid)

In [ ]:
def svm_feats(data, cat, imp, scale = 's'):
    
    """
    Funtion for looping over makesvm() using more and more features in feature list("imp").
    
    Input:
    data(n x m pd DataFrame): X-varaible; datamatrix with features as columns and datapoints as rows.
    cat(n-lenght list-like): Y-variable; labels to classify after
    imp(array): list of significant features found in X, ranked from most important to least.
    scale(string, "s", "p" or "n") :scaling method. "s" = standard(default). "p" = Pareto. "n" = No scaling.
    Output:
    svm_results(DataFrame): Classification succes rate of each feature set
    
    """
    
    #tell user what scaling method they have chosen
    if scale == 's':
        print("Scaling method: Auto-scale")
    elif scale == 'p':
        print("Scaling method: Pareto")
    elif scale == 'n':
        print("Scaling method: None... Are you sure about this, champ?")
    else:
        raise Exception("No correct scale method specified: 'scale' has to be 's','p' or 'n'")
    
    svm_results = pd.DataFrame(columns= ["score", "params"]) 
    
    #loop through feature list
    for i in np.arange(len(imp)):
        
        #include features up to and with i
        feat = imp[:i+1]
        bob = makesvm(data[feat], cat, scale)
        
        #save stats
        s_i = bob.best_score_
        p_i = bob.best_params_
        svm_results.loc[i+1] = [s_i, p_i]
    
    #once more with full featureset
    bob = makesvm(data, cat, scale)
    
    s_i = bob.best_score_
    p_i = bob.best_params_
    svm_results.loc['Full'] = [s_i, p_i]
    
    return(svm_results)


In [ ]:
def bin_data(d, bl = 0.02):
    """
    Sums data into bins of uniform lenght and plots an overview of the binned data.
    
    Input:
    d(DataFrame): Data to be strutured into bins, with ppm values, in decending order as column names.
    bl(float): Length of the bins. Default is 0.02.
    Output:
    binned_d(DataFrame): d summed into bins, with columns named for lowest bin, roudend to three ciffers. 
    
    """
    #make bins
    bins = np.arange(d.columns.values[0], d.columns.values[-1], -bl)
    
    #define start of first bin
    left = bins[0]
    
    #dataframe for output
    binned_d = pd.DataFrame(index= data.index)
    
    #loop over all bins
    for b in bins[1:]:
        #columns in original data to inlcude in this bin
        d_b = d[d.columns[(left >= d.columns) & (d.columns > b)]]
        
        #round bin name
        b = np.round(b,3)
        
        #set sum of original data as values for this bin
        binned_d[b] = d_b.sum(axis=1)
        
        #define start point of next bin
        left = b
    
    print("There are %i bins in total" %(len(binned_d.columns)))
    
    #make plot of binned data
    plt.rcParams['figure.figsize'] = [40, 20]
    
    ax = binned_d.T.plot(legend= None)
    xtick = bins[1:]
    ax.set_xticks( xtick, minor=True )
    ax.grid(True, which='minor', axis='x' ) #Show bins as grid
    ax.grid(False, which='major', axis='x' ) #has to be False or extra grid lines, not showing bins
    #couldn't get grid to show over numbers
    
    return(binned_d)